# Dual BLE: Heart Rate + Sterzo Example

This is an advanced demonstration that builds on [`single_hr_notebook_example.ipynb`](single_hr_notebook_example.ipynb) to demonstrate streaming from two devices simultaneously.

This examples uses Bluetooth Low Energy (BLE) Heart Rate service and BLE Sterzo.
+ It must support Bluetooth, not just ANT+
+ The GATT Characteristic required is: `00002a37-0000-1000-8000-00805f9b34fb`
+ You may use the [`bluetooth_dissect.py` tool](https://github.com/tensorturtle/bluetooth_dissect) to check if your device supports that GATT Characteristic.

Example compatible Heart Rate devices:
+ Magene H603
+ Wahoo TICKR & TICKR Fit
+ WHOOP 4.0 (Broadcast Heart Rate ON)
+ and many more advertised as "supports Bluetooth Heart Rate"

Compatible Sterzo device:
+ [Elite Sterzo Smart Steering Plate](https://www.elite-it.com/en/products/home-trainers/ecosystem-accessories/sterzo-smart)

In [1]:
!pip3 install -r -q requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '-q'
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Setup

In [2]:
import asyncio

In [3]:
from bleak_fsm import machine, BleakModel

In [4]:
hr_model = BleakModel()
sterzo_model = BleakModel()

machine.add_model(hr_model)
machine.add_model(sterzo_model)

Toggle this `USE_PYCYCLING` flag to see how this library can accomodate using standard, raw BLeakClient or Pycycling-wrapped BleakClient.

In [5]:
# Define callbacks

from pycycling.heart_rate_service import HeartRateService
hr_model.wrap = lambda client: HeartRateService(client)
hr_model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
hr_model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
def handle_hr_measurement(value):
    print("Using Pycycling wrapper around BleakClient")
    print(f"Heart Rate: {value}")
hr_model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)

from pycycling.sterzo import Sterzo
sterzo_model.wrap = lambda client: Sterzo(client)
sterzo_model.enable_notifications = lambda client: client.enable_steering_measurement_notifications()
sterzo_model.disable_notifications = lambda client: client.disable_steering_measurement_notifications()
def handle_steer_measurement(value):
    print("Using Pycycling wrapper around BleakClient")
    print(f"Sterzo: {value}")
sterzo_model.set_measurement_handler = lambda client: client.set_steering_measurement_callback(handle_steer_measurement)


# Usage

We can use any of the two models to call `start_scan()`, and the results will be shared among them through their shared class variable.

In [6]:
await BleakModel.start_scan()

True

In [7]:
await asyncio.sleep(3)

In [8]:
await BleakModel.stop_scan()

True

In [9]:
BleakModel.bt_devices

{'14863C4D-BF71-4EA3-C6B4-98001056AAF8': (BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
  AdvertisementData(local_name='WHOOPDEDOO', service_uuids=['0000180d-0000-1000-8000-00805f9b34fb'], rssi=-50)),
 '37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4': (BLEDevice(37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4, GiGA Genie 3_F70E),
  AdvertisementData(local_name='GiGA Genie 3_F70E', manufacturer_data={65535: b'\x00\x15C\x11\xff\xf0\x11\xe3D9\xbb\x12\xac\xdc2\x14\x00\x00\x00\xfa\xf7\x0e\xc5'}, rssi=-77)),
 '715D8603-DC4C-2994-C0CD-2BC5A93E0B38': (BLEDevice(715D8603-DC4C-2994-C0CD-2BC5A93E0B38, STERZO),
  AdvertisementData(local_name='STERZO', service_uuids=['347b0001-7635-408b-8918-8ff3949ce592'], rssi=-74)),
 '9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41': (BLEDevice(9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41, Study),
  AdvertisementData(manufacturer_data={76: b'\x0f\x05\x90\x00\xa5=\x14\x10\x02-\x04'}, tx_power=6, rssi=-52)),
 'DC2343A7-C46E-E1BB-30E9-BC6E274B4A39': (BLEDevice(DC2343A7-C46E-E1BB-30E9-BC6E2

Copy-and-paste the key corresponding to the device that you want to connect to:

In [10]:
hr_target_address = "14863C4D-BF71-4EA3-C6B4-98001056AAF8"
sterzo_target_address = "715D8603-DC4C-2994-C0CD-2BC5A93E0B38"

In [11]:
hr_model.state

'Init'

In [12]:
sterzo_model.state

'Init'

In [13]:
await hr_model.set_target(hr_target_address)
await sterzo_model.set_target(sterzo_target_address)

True

In [14]:
hr_model.state

'TargetSet'

In [15]:
sterzo_model.state

'TargetSet'

In [16]:
await hr_model.connect()

True

In [17]:
await sterzo_model.connect()

True

Sidebar: Note how the above cell returns `True`. All transition methods on `model` returns True if the transition was successful. Therefore you may explicitly define happy-path and exception-path in different ways.

For debugging,
```python
assert await model.connect(), "Model failed to connect"
```

Or, more properly check the return value
```python
success = await model.connect()
if not succcess:
    await model.clean_up() # this catch-all method brings the model back to "Init"
    raise Exception("Failed to connect")
```

Or, we can check the `.state` afterwards, like so:

In [18]:
if hr_model.state != "Connected" or sterzo_model.state != "Connected":
    raise Exception("Failed to connect. If the device was not properly disconnected last time, restart either the device or bluetooth adapter")

In order to receive/send actual data from/to the connected device, we need to start a stream.

This will run forever until `model.disconnect()` is called, so for now let's sleep for several seconds and then call that.

In [19]:
await hr_model.stream()

True

In [20]:
await sterzo_model.stream()

True

In [21]:
await asyncio.sleep(10)

Using Pycycling wrapper around BleakClient
Sterzo: nan
Using Pycycling wrapper around BleakClient
Sterzo: 0.736236572265625
Using Pycycling wrapper around BleakClient
Sterzo: 0.9089202880859375
Using Pycycling wrapper around BleakClient
Sterzo: 0.8070755004882812
Using Pycycling wrapper around BleakClient
Sterzo: 1.2215652465820312
Using Pycycling wrapper around BleakClient
Sterzo: 0.7060928344726562
Using Pycycling wrapper around BleakClient
Sterzo: 0.492279052734375
Using Pycycling wrapper around BleakClient
Sterzo: 0.9089202880859375
Using Pycycling wrapper around BleakClient
Sterzo: 0.8070755004882812
Using Pycycling wrapper around BleakClient
Sterzo: 0.9793319702148438
Using Pycycling wrapper around BleakClient
Sterzo: 0.46384429931640625
Using Pycycling wrapper around BleakClient
Sterzo: 0.8774948120117188
Using Pycycling wrapper around BleakClient
Sterzo: 0.8774948120117188
Using Pycycling wrapper around BleakClient
Sterzo: 0.5351104736328125
Using Pycycling wrapper around Bleak

Bleak-FSM encapsulates a kind of bug in Bleak, where it is not possible to re-use a connected client. 

That is, we shouldn't step back to "Connected" state from "Streaming", because we can't go from "Connected" back to "Streaming".

Instead we need to go back two steps to "TargetSet", re-establish a connection, and then we can stream again.

Bleak-FSM enforces this behavior for your own safety.
There exists not method for you to even try to go from "Streaming" to "Connected".


In [22]:
await hr_model.clean_up()

Using Pycycling wrapper around BleakClient
Sterzo: 0.8070755004882812
Using Pycycling wrapper around BleakClient
Sterzo: 0.39215087890625
Using Pycycling wrapper around BleakClient
Sterzo: 0.6352462768554688
Using Pycycling wrapper around BleakClient
Sterzo: 1.04931640625
Using Pycycling wrapper around BleakClient
Sterzo: 0.5639801025390625
Using Pycycling wrapper around BleakClient
Sterzo: 0.5351104736328125
Using Pycycling wrapper around BleakClient
Sterzo: 0.8774948120117188
Using Pycycling wrapper around BleakClient
Sterzo: 1.2215652465820312
Using Pycycling wrapper around BleakClient
Sterzo: 1.0170669555664062
Using Pycycling wrapper around BleakClient
Sterzo: 1.1880340576171875
Using Pycycling wrapper around BleakClient
Sterzo: 0.6352462768554688
Using Pycycling wrapper around BleakClient
Sterzo: 0.8380889892578125
Using Pycycling wrapper around BleakClient
Sterzo: 0.39215087890625
Using Pycycling wrapper around BleakClient
Sterzo: 0.9089202880859375
Using Pycycling wrapper aroun

True

In [23]:
await sterzo_model.clean_up()

True

Whenever the application terminates, or otherwise no longer needs to communicate with the bluetooth device, it should always call `disconnect()` on the model. The model state should always e either "TargetSet" or "Init".